# Color dithering

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import ticker

import numpy as np

import astro3d
from astro3d.image_stack import color_replace

Make up some fake data: two gaussian bumps

In [ ]:
x = np.linspace(-2, 2, 1000)
y = np.linspace(-2, 2, 990)
X,Y = np.meshgrid(x, y, indexing='ij')

# peak density
rho0 = 1e0
dyn_range = 1e-3

data = rho0 * (
    1.0 * np.exp(- ((X-0.5)**2 + (Y-0.5)**2)/(2*0.5**2)) +
    0.1 * np.exp(- ((X+0.5)**2 + (Y+0.5)**2)/(2*0.5**2))
    )

# Define norm and plot the data

In [ ]:
vmax = data.max()
norm = LogNorm(dyn_range * vmax, vmax)

# plot the data

f, ax = plt.subplots()
ax.set_aspect('equal')
cc=ax.pcolormesh(x, y, data.T, norm=norm)
plt.colorbar(cc);

Define three densities we want to color, along with the scatter around that density value in dex. Fill is the filling factor to scale some colors up or down.

In [ ]:
levels = np.array([0.15, 0.5, 0.7, 0.9])
sigmas = np.array([0.1, 0.05, 0.03, 0.03])
fill = np.array([0.5, 0.5, 0.5, 0.5])

apply the norm to the data

In [ ]:
layer_norm = np.array(norm(data))

Show a histogram of the data values

In [ ]:
astro3d.image_stack.image_stack.sh

In [ ]:
bins = np.linspace(0, 1, 100)
counts, _ = np.histogram(layer_norm.ravel(), bins=bins)

In [ ]:
f, ax = plt.subplots()
ax.bar(bins[:-1], counts, align='edge', width=np.diff(bins), alpha=0.5)
ax.set_xlim(0, 1)
ax2 = ax.secondary_xaxis('top', functions=(norm.inverse, norm))
for i, (_level, _sig) in enumerate(zip(levels, sigmas)):
    line = ax.axvline(_level, ls='--', c=f'C{i}')
    ax.errorbar(_level, np.mean(ax.get_ylim()) * (1 + 0.1 * i),
                xerr=[
                    [_sig],
                    [_sig]], c=line.get_color(), capsize=5)
    
ax2.get_xaxis().set_major_locator(ticker.LogLocator())

ax.set_xlabel('original density')
ax2.set_xlabel('normalized density');

for each color, we compute the distance-image, i.e. how far away we are from the desired contour.

We use the weighted square-distance in normalized space for each pixel value $D_{k,l}$. Each color, which is specified by its normalized density level $N_i$ and the width around that density value in dex, $\sigma_i$
$$
d^2_{k,l,i} = \frac{\left(N _i - D_{k, l}\right)^2}{2\, \sigma_i^2}
$$

The amount of color is then assigned according to that distance:

$$\mathrm{color}_i = \frac{1}{1 + d^2 _ {k,l,i}}$$

In [ ]:
# dist_sq = (np.log10(np.array(rho_i)[None, None, :] / data[..., None]) / sig)**2
dist_sq = (np.array(levels)[None, None, :] - layer_norm[..., None])**2 / (2 * sigmas**2)
color_density = 1 / (1 + dist_sq) * fill

In [ ]:
example_fills = [0.1, 0.5, 1]

f, ax = plt.subplots(len(example_fills), len(levels) + 1, dpi=100, figsize=(4 * (len(levels) + 1), 4 * len(example_fills)))

for i, _fill in enumerate(example_fills):
    
    _color_density = 1 / (1 + dist_sq) * _fill
    im_cd = astro3d.fmodule.dither_colors(_color_density)
    
    for j, _level in enumerate(levels):
        ax[i,j].imshow(im_cd[:,:,j], origin='lower', vmin=0, vmax=1, cmap='gray_r')
        ax[i,j].set_title(f'component {j}')
        
    ax[i, -1].imshow(_color_density.sum(-1), origin='lower', vmin=0, vmax=1, cmap='gray_r')
    ax[i, -1].set_title('un-dithered')
    ax[i, -1].set_ylabel(f'fill = {_fill:.2g}')
    ax[i, -1].yaxis.set_label_position("right")

Now for our chosen values of `fill`

In [ ]:
im_cd = astro3d.fmodule.dither_colors(color_density * fill)

f, axs = plt.subplots(1, len(levels)+1, figsize=(3 * (len(levels) + 1), 3), dpi=150)
args = {'origin':'lower', 'cmap':'gray_r'}

for i in range(im_cd.shape[-1]):
    axs[i].imshow(im_cd[:, :, i], **args)
    axs[i].set_title(f'component {i}')

    axs[-1].imshow(im_cd.sum(-1), **args)

print(f'there is {"no" if im_cd.max(-1).max()==1 else ""} overlap')

# Replacing Colors

So far we worked with 3 unique RGB colors. For the printing, we have, however only the 3 base-colors VeroCyan, VeroMagenta, and VeroYellow available. We can either use those or we could try to mix them to another color, for example by replacing 50% of the currently red pixels with Magenta, and 50% with Yellow. The mixing on the Stratasys Pallete between C, M, and Y looks different from just a linear interpolation.

In [ ]:
from astro3d.image_stack import VeroC_sRGB, VeroM_sRGB, VeroT_sRGB, VeroY_sRGB

## Simple case:
replacing colors direclty (doesn't really make a difference as the material is assigned during printing by hand).

In [ ]:
im = im_cd.copy()

old_colors = np.eye(len(levels))
old_colors = np.vstack((old_colors, np.zeros(old_colors.shape[1])))

new_colors = [VeroM_sRGB, VeroY_sRGB, VeroC_sRGB, 0.5 * (VeroM_sRGB + VeroC_sRGB), VeroT_sRGB]

im_stack = []

for col_o, col_n in zip(old_colors, new_colors):
    im_stack += [color_replace(im, col_o, col_n)]

im = np.array(im_stack).sum(0)

#check that we didn't mess up the colors
_cols = np.unique(im.reshape(-1, 3), axis=0)
print(f'there are {len(_cols)} color(s) in the image')
f, ax = plt.subplots()
ax.imshow([_cols])
ax.axis('off')

f, ax = plt.subplots(1,2, dpi=150)
ax[0].imshow(im_cd.sum(-1), cmap='gray_r', origin='lower')
ax[1].imshow(im, origin='lower');

## Interesting case:
replacing one color by a color mix

In [ ]:
im = im_cd.copy()

new_colors = [
    [VeroY_sRGB, VeroC_sRGB],
    [VeroM_sRGB, VeroC_sRGB],
    [VeroM_sRGB, VeroY_sRGB],
    [VeroM_sRGB, VeroY_sRGB],
    [VeroT_sRGB]
    ]

mixes = [
    [0.2, 0.8],
    [0.2, 0.8],
    [0.5, 0.5],
    [0.2, 0.8],
    [1]
    ]

im_stack = []

for col_o, col_n, _f in zip(old_colors, new_colors, mixes):
    im_stack += [color_replace(im, col_o, col_n, f=_f, inplace=False)]

im = np.array(im_stack).sum(0)


# check that we didn't mess up the colors
_cols = np.unique(im.reshape(-1, 3), axis=0)
print(f'there are {len(_cols)} color(s) in the image')
f, ax = plt.subplots()
ax.imshow([_cols])
ax.set_xticks([])
ax.set_yticks([])

f, ax1 = plt.subplots(1,2, dpi=150)
ax1[0].imshow(im_cd.sum(-1), cmap='gray_r', origin='lower')
ax1[1].imshow(im, origin='lower');

In [ ]:
import imageio
imageio.imsave('test.png', np.uint8(255 * im))

## Same with just one call

In [ ]:
from astro3d.image_stack.image_stack import makeslice_color

In [ ]:
makeslice_color(0, [0], lambda coords: np.array([data]).T,
                np.array([[1],[2],[3]]), norm, './',
                levels=levels, sigmas=sigmas, fill=fill,
                colors=new_colors, f=mixes)

In [ ]:
f, ax = plt.subplots(dpi=150)
ax.imshow(imageio.imread('slice_0000.png'), origin='lower');

In [ ]:
N = 100

a = np.zeros([2 * N, 4 * N, 3])

a[:, 0 * N:1 * N, 0] = 1.
a[:, 1 * N:2 * N, 1] = 1.
a[:, 2 * N:3 * N, 2] = 1.

a[:, 3 * N:, 0] = 1.0
a[:, 3 * N:, 1] = 0.5
a[:, 3 * N:, 2] = 1.0

In [ ]:
b = astro3d.fmodule.dither_colors(a)

In [ ]:
f, ax = plt.subplots(1, 2)
ax[0].imshow(a)
ax[1].imshow(b)